In [1]:
from pynvml import smi

In [22]:
nvsmi = smi.nvidia_smi.getInstance()

In [23]:
nvsmi

In [24]:
result = nvsmi.DeviceQuery('memory.free, memory.total, power.draw, power.limit')

In [25]:
type(result)

dict

In [26]:
print(result)

{'gpu': [{'fb_memory_usage': {'total': 16160.5, 'free': 16160.375, 'unit': 'MiB'}, 'power_readings': {'power_draw': 23.406, 'power_limit': 300.0, 'power_state': 'P0', 'unit': 'W'}}]}


In [27]:
nvsmi.DeviceQuery()['gpu'][0]

{'id': '0000:00:04.0',
 'product_name': 'Tesla V100-SXM2-16GB',
 'product_brand': 'Tesla',
 'display_mode': 'Enabled',
 'display_active': 'Disabled',
 'persistence_mode': 'Disabled',
 'accounting_mode': 'Disabled',
 'accounting_mode_buffer_size': '4000',
 'driver_model': {'current_dm': 'N/A', 'pending_dm': 'N/A'},
 'serial': '0322817023286',
 'uuid': 'GPU-3c87c494-d675-61b6-f150-0d2bde251ef7',
 'minor_number': '0',
 'vbios_version': '88.00.4F.00.09',
 'multigpu_board': 'No',
 'board_id': '0x4',
 'inforom_version': {'img_version': 'G503.0201.00.03',
  'oem_object': '1.1',
  'ecc_object': '5.0',
  'pwr_object': 'N/A'},
 'gpu_operation_mode': {'current_gom': 'N/A', 'pending_gom': 'N/A'},
 'pci': {'pci_bus': '00',
  'pci_device': '04',
  'pci_domain': '0000',
  'pci_device_id': '1DB110DE',
  'pci_bus_id': '0000:00:04.0',
  'pci_sub_system_id': '121210DE',
  'pci_gpu_link_info': {'pcie_gen': {'max_link_gen': '3',
    'current_link_gen': '3'},
   'link_widths': {'max_link_width': '16x', 'cur

In [33]:
nvsmi.DeviceQuery([smi.NVSMI_UTILIZATION_GPU, smi.NVSMI_UTILIZATION_MEM, smi.NVSMI_POWER_DRAW, smi.NVSMI_POWER_LIMIT])

{'gpu': [{'utilization': {'gpu_util': 0, 'memory_util': 0, 'unit': '%'},
   'power_readings': {'power_draw': 23.436,
    'power_limit': 300.0,
    'power_state': 'P0',
    'unit': 'W'}}]}

In [ ]:
def report_gpu_metric(value, type, instance_id, zone, project_id):
    series = monitoring_v3.types.TimeSeries()
    series.metric.type = 'custom.googleapis.com/{type}'.format(type=type)
    series.resource.type = 'gce_instance'
    series.resource.labels['instance_id'] = instance_id
    series.resource.labels['zone'] = zone
    series.resource.labels['project_id'] = project_id
    point = series.points.add()
    point.value.int64_value = value
    now = time.time()
    point.interval.end_time.seconds = int(now)
    point.interval.end_time.nanos = int(
        (now - point.interval.end_time.seconds) * 10**9)
    client.create_time_series(project_name, [series])

In [180]:
from google.cloud import monitoring_v3
from google.cloud.monitoring_v3.query import Query
import datetime
import time

In [181]:
client = monitoring_v3.MetricServiceClient()
project_name = f'projects/jk-mlops-dev'
filter = 'metric.type = starts_with("custom.googleapis.com/opencensus")'


In [182]:
for descriptor in client.list_metric_descriptors(name=project_name, filter_=filter):
    print(f'Descriptor: {descriptor.name}')
    print(descriptor)

Descriptor: projects/jk-mlops-dev/metricDescriptors/custom.googleapis.com/opencensus/gce/gpu/memory_utilization_distribution
name: "projects/jk-mlops-dev/metricDescriptors/custom.googleapis.com/opencensus/gce/gpu/memory_utilization_distribution"
labels {
  key: "device"
}
metric_kind: CUMULATIVE
value_type: DISTRIBUTION
unit: "%"
description: "The distribution of gpu memory utilization"
display_name: "OpenCensus/gce/gpu/memory_utilization_distribution"
type: "custom.googleapis.com/opencensus/gce/gpu/memory_utilization_distribution"

Descriptor: projects/jk-mlops-dev/metricDescriptors/custom.googleapis.com/opencensus/gce/gpu/utilization_distribution
name: "projects/jk-mlops-dev/metricDescriptors/custom.googleapis.com/opencensus/gce/gpu/utilization_distribution"
labels {
  key: "device"
}
metric_kind: CUMULATIVE
value_type: DISTRIBUTION
unit: "%"
description: "The distribution of gpu utilization"
display_name: "OpenCensus/gce/gpu/utilization_distribution"
type: "custom.googleapis.com/ope

In [183]:
filter = 'metric.type = "custom.googleapis.com/opencensus/gce/gpu/utilization_distribution" AND metric.labels.device = "0"'

now = time.time()
seconds = int(now)
nanos = int((now - seconds)*10**9)
interval = monitoring_v3.types.TimeInterval(
        end_time = {"seconds": seconds, "nanos": nanos},
        start_time = {"seconds": (seconds - 120), "nanos": nanos}
)

In [184]:
results = client.list_time_series(
        name=project_name,
        filter_=filter,
        interval=interval,
        view=monitoring_v3.types.ListTimeSeriesRequest.TimeSeriesView.FULL
)

In [185]:
for result in results:
   print(result)

metric {
  labels {
    key: "device"
    value: "0"
  }
  type: "custom.googleapis.com/opencensus/gce/gpu/utilization_distribution"
}
resource {
  type: "gce_instance"
  labels {
    key: "instance_id"
    value: "7148291146842715683"
  }
  labels {
    key: "project_id"
    value: "jk-mlops-dev"
  }
  labels {
    key: "zone"
    value: "us-central1-a"
  }
}
metric_kind: CUMULATIVE
value_type: DISTRIBUTION
points {
  interval {
    start_time {
      seconds: 1611619512
      nanos: 432710000
    }
    end_time {
      seconds: 1611619632
      nanos: 688584000
    }
  }
  value {
    distribution_value {
      count: 57
      mean: 97.78947368421055
      sum_of_squared_deviation: 11.473684210526368
      bucket_options {
        explicit_buckets {
          bounds: 0.0
          bounds: 11.0
          bounds: 21.0
          bounds: 31.0
          bounds: 41.0
          bounds: 51.0
          bounds: 61.0
          bounds: 71.0
          bounds: 81.0
          bounds: 91.0
         

In [9]:
dir(monitoring_v3.types)

['Aggregation',
 'AlertPolicy',
 'Any',
 'BasicSli',
 'BoolValue',
 'BytesValue',
 'CreateAlertPolicyRequest',
 'CreateGroupRequest',
 'CreateMetricDescriptorRequest',
 'CreateNotificationChannelRequest',
 'CreateServiceLevelObjectiveRequest',
 'CreateServiceRequest',
 'CreateTimeSeriesError',
 'CreateTimeSeriesRequest',
 'CreateTimeSeriesSummary',
 'CreateUptimeCheckConfigRequest',
 'DeleteAlertPolicyRequest',
 'DeleteGroupRequest',
 'DeleteMetricDescriptorRequest',
 'DeleteNotificationChannelRequest',
 'DeleteServiceLevelObjectiveRequest',
 'DeleteServiceRequest',
 'DeleteUptimeCheckConfigRequest',
 'Distribution',
 'DistributionCut',
 'DoubleValue',
 'DroppedLabels',
 'Duration',
 'Empty',
 'FieldMask',
 'FloatValue',
 'GetAlertPolicyRequest',
 'GetGroupRequest',
 'GetMetricDescriptorRequest',
 'GetMonitoredResourceDescriptorRequest',
 'GetNotificationChannelDescriptorRequest',
 'GetNotificationChannelRequest',
 'GetNotificationChannelVerificationCodeRequest',
 'GetNotificationChann

In [85]:
for descriptor in client.list_metric_descriptors(name=project_name, filter_=filter):
    print(f'Deleting: {descriptor.name}')
    client.delete_metric_descriptor(name=descriptor.name)

In [87]:
project_id = 'jk-mlops-dev'
metric_type = 'custom.googleapis.com/opencensus/gce/gpu/utilization_distribution'

In [88]:
START_TIME = datetime.datetime.fromisoformat('2021-01-22T04:00:00-00:00')
END_TIME = datetime.datetime.fromisoformat('2021-01-22T05:00:00-00:00')

In [91]:
query = Query(client, project_id, metric_type=metric_type)
query = query.select_interval(END_TIME, START_TIME)
#query = query.select_resources(instance_id='1297153725150009955')
#query = query.select_metrics(opencensus_task='py-13989@jk-test')

In [94]:
df = query.as_dataframe()
df

resource_type                                                    gce_instance  \
project_id                                                       jk-mlops-dev   
zone                                                               us-west1-b   
instance_id                                               5382047594078986292   
device                                                                      0   
2021-01-22 04:02:11.137948  count: 6\nbucket_options {\n  explicit_buckets...   
2021-01-22 04:02:41.138088  count: 12\nbucket_options {\n  explicit_bucket...   
2021-01-22 04:03:11.138221  count: 18\nbucket_options {\n  explicit_bucket...   
2021-01-22 04:03:41.138345  count: 24\nbucket_options {\n  explicit_bucket...   
2021-01-22 04:04:11.138576  count: 30\nbucket_options {\n  explicit_bucket...   
2021-01-22 04:04:41.138740  count: 36\nbucket_options {\n  explicit_bucket...   
2021-01-22 04:05:11.138878  count: 42\nbucket_options {\n  explicit_bucket...   

resource_type                                                                  
project_id                                                                     
zone                                                                           
instance_id                                                                    
device                                                                      1  
2021-01-22 04:02:11.137948  count: 6\nbucket_options {\n  explicit_buckets...  
2021-01-22 04:02:41.138088  count: 12\nbucket_options {\n  explicit_bucket...  
2021-01-22 04:03:11.138221  count: 18\nbucket_options {\n  explicit_bucket...  
2021-01-22 04:03:41.138345  count: 24\nbucket_options {\n  explicit_bucket...  
2021-01-22 04:04:11.138576  count: 30\nbucket_options {\n  explicit_bucket...  
2021-01-22 04:04:41.138740  count: 36\nbucket_options {\n  explicit_bucket...  
2021-01-22 04:05:11.138878  count: 42\nbucket_options {\n  explicit_bucket...